In [1]:
import numpy as np
import time
import itertools
import numba
import math



In [2]:
# use this cell to define functions

def make_strats(fbids, nagents, ntags, prop_of_types):
    
    fstrategies = np.random.choice(fbids, [nagents, ntags*2])

    # print(strategies0)

    # fill in the tags column with the tags
    # fill in the tags+1 column with the groups
    oldprop = 0 
    propcount = 0 
    for idx00 in prop_of_types:
        newprop = int(idx00*nagents)
        for idx01 in range(0, newprop):

            fstrategies[oldprop+idx01, ntags] = propcount
            fstrategies[oldprop+idx01, ntags+1] = propcount

        oldprop = newprop
        propcount += 1

    return fstrategies



@numba.jit
def ppi_payoffs(fzeros0, fstrats0, fnagents0, fbreakdowns0):
    
    fpayoffs0 = fzeros0
    
    for idx10 in range(0, fnagents0):
        
        paystrat = fstrats0[idx10]
        
        paystrattag = math.floor(paystrat[-2])
        
        paygroup = math.floor(paystrat[-1])
        
        for idx11 in range(1, fnagents0):
            
            pair = (idx10+idx11)%fnagents0
            
            pairstrat = fstrats0[pair]
            
            pairstrattag = math.floor(pairstrat[-2])
            
            totalbid = paystrat[pairstrattag]+pairstrat[paystrattag]
            
            if totalbid <= 1:
                fpayoffs0[idx10] = fpayoffs0[idx10] + paystrat[pairstrattag]
            else:
                fpayoffs0[idx10] = fpayoffs0[idx10] + fbreakdowns0[paygroup]
                
    return fpayoffs0


# note that random pairings only avoids mapping an agent to itself if number of agents is divisible by 4
@numba.jit
def ppi_update_withtags(fstrats1, fnagents1, fum, fpairings00, fpairings11, frandunif00, fpayoffs1):
    
    fnewstrats = fstrats1.copy()
    
    for idx20 in range(0, fnagents1):
        
        leftstrat = fstrats1[fpairings00[idx20]]
        rightstrat = fstrats1[fpairings11[-(idx20+1)]]
        
        leftpay = fpayoffs1[fpairings00[idx20]]
        rightpay = fpayoffs1[fpairings11[-(idx20+1)]]
        
        leftrandum = (rightpay-leftpay)/fum
        
        leftrand = frandunif00[fpairings00[idx20]]

        
        if (leftpay < rightpay) & (leftrand < leftrandum):
            fnewstrats[fpairings00[idx20]][0:3] = rightstrat[0:3]

            
    return fnewstrats


# note that random pairings only avoids mapping an agent to itself if number of agents is divisible by 4
@numba.jit
def ppi_update_notags(fstrats1, fnagents1, fum, fpairings00, fpairings11, frandunif00, fpayoffs1):
    
    fnewstrats = fstrats1.copy()
    
    for idx20 in range(0, fnagents1):
        
        leftstrat = fstrats1[fpairings00[idx20]]
        rightstrat = fstrats1[fpairings11[-(idx20+1)]]
        
        leftpay = fpayoffs1[fpairings00[idx20]]
        rightpay = fpayoffs1[fpairings11[-(idx20+1)]]
        
        leftrandum = (rightpay-leftpay)/fum
        
        leftrand = frandunif00[fpairings00[idx20]]

        
        if (leftpay < rightpay) & (leftrand < leftrandum):
            fnewstrats[fpairings00[idx20]][0:2] = rightstrat[0:2]

            
    return fnewstrats




# def random10step(frng, fnagents2):
    
#     fpairings0 = np.zeros((10, fnagents2), dtype=int)
#     fpairings1 = np.zeros((10, fnagents2), dtype=int)
    
#     for idx30 in range(0, 10):
#         leftpairings = frng.permutation(fnagents2//2)
#         rightpairings = frng.permutation(fnagents2//2) + fnagents2//2
        
#         fpairings0[idx30] = np.append(leftpairings, rightpairings)
#         fpairings1[idx30] = np.append(rightpairings, leftpairings)
        
    
#     frandunif0 = frng.random((10, fnagents2))
    
#     frandunif2 = frng.random((10, fnagents2))
    
#     frandunif4 = frng.random((10, fnagents2))
    
#     frandunif6 = frng.random((10, fnagents2))
    
#     frandunif8 = frng.random((10, fnagents2))
    
#     fzeros2 = np.zeros((10, fnagents2), dtype = np.float64)
    
#     return fzeros2, fpairings0, fpairings1, frandunif0, frandunif2, frandunif4, frandunif6, frandunif8

def random10step(frng, fnagents2, typesprop2):
    
    fpairings0 = np.zeros((10, fnagents2), dtype=int)
    fpairings1 = np.zeros((10, fnagents2), dtype=int)
    
    foutpairings = np.zeros((10, fnagents2), dtype=int)
    
    leftnum = int(fnagents2*typesprop2[0])
    rightnum = int(fnagents2*typesprop2[1])
    
    for idx30 in range(0, 10):
        leftpairings = frng.permutation(leftnum)
        rightpairings = frng.permutation(rightnum) + leftnum
        
        fpairings0[idx30] = np.append(leftpairings, rightpairings)
        fpairings1[idx30] = np.append(rightpairings, leftpairings)
        
        leftout = frng.choice(rightnum, leftnum) + leftnum
        rightout = frng.choice(leftnum, rightnum)
        
        foutpairings[idx30] = np.append(leftout, rightout)
    
    frandunif0 = frng.random((10, fnagents2))
    
    frandunif2 = frng.random((10, fnagents2))
    
    frandunif4 = frng.random((10, fnagents2))
    
    frandunif6 = frng.random((10, fnagents2))
    
    frandunif8 = frng.random((10, fnagents2))
    
    frandunif10 = frng.random((10, fnagents2))
    
    fzeros2 = np.zeros((10, fnagents2), dtype = np.float64)
    
    return fzeros2, fpairings0, fpairings1, frandunif0, frandunif2, frandunif4, frandunif6, frandunif8, frandunif10, foutpairings


@numba.jit
def ppi_mutation_withTags(fstrats3, fmutprob3, frandunif22, frandunif44, frandunif66, frandunif88, fbids3, fnagents3):
    
    flow = fbids3[0]
    fmed = fbids3[1]
    fhigh = fbids3[2]
    
    for idx40 in range(0, fnagents3):
        
        if frandunif22[idx40] < fmutprob3:
            newstrat3 = (fstrats3[idx40]).copy()
            if frandunif44[idx40] < 1/3:
                newstrat3[0] = flow
            elif frandunif44[idx40] < 2/3:
                newstrat3[0] = fmed
            else:
                newstrat3[0] = fhigh
            
            if frandunif66[idx40] < 1/3:
                newstrat3[1] = flow
            elif frandunif66[idx40] < 2/3:
                newstrat3[1] = fmed
            else:
                newstrat3[1] = fhigh
            
            if frandunif88[idx40] < 0.5:
                newstrat3[2] = 0
            else:
                newstrat3[2] = 1
            
            fstrats3[idx40] = newstrat3
    
    return fstrats3

    
    
@numba.jit
def MATCHcount_disc(fstrats4, fhigh4):
    count40 = 0
    count41 = 0
    
    for idx60 in range(0, len(fstrats4)):
        for idx61 in range(1, len(fstrats4)):
            IDstrat0 = fstrats4[idx60]
            IDstrat1 = fstrats4[((idx60+idx61)%len(fstrats4))]
            
            IDstrattag0 = math.floor(IDstrat0[-2])
            IDstrattag1 = math.floor(IDstrat1[-2])
            
            if IDstrattag0 != IDstrattag1:
                count40 +=1
                if IDstrat0[IDstrattag1] == fhigh4:
                    count41 +=1
    if count40 == 0:
        finalIDcount = 0
    else:
        finalIDcount = count41/count40
    return finalIDcount

@numba.jit
def count_avg_pay(fstrats5, fpay5):
    count50a = 0
    count50b = 0
    count51a = 0
    count51b = 0
    
    for idx70 in range(0, len(fstrats5)):
        strat50 = fstrats5[idx70]
        pay50 = fpay5[idx70]
        
        if strat50[-1] == 0:
            count50a += 1
            count50b += pay50#/len(fstrats5)
        elif strat50[-1] == 1:
            count51a += 1
            count51b += pay50#/len(fstrats5)
            
    count50c = count50b/(count50a)
    count51c = count51b/(count51a)
    
    return count50c, count51c


@numba.jit
def count_avg_payAGAINSTout(fzeros60, fstrats60, fnagents60, fbreakdowns60):
    
    fpayoffs60 = fzeros60
    
    for idx80 in range(0, fnagents60):
        count60b = 0
        count60a = 0
        
        paystrat = fstrats60[idx80]
        
        paystrattag = math.floor(paystrat[-2])
        
        paygroup = math.floor(paystrat[-1])
        
        for idx81 in range(1, fnagents60):
            
            pair = (idx80+idx81)%fnagents60
            
            pairstrat = fstrats60[pair]
            
            pairstrattag = math.floor(pairstrat[-2])
            
            pairgroup = math.floor(pairstrat[-1])
            
            totalbid = paystrat[pairstrattag]+pairstrat[paystrattag]
            
            if paygroup != pairgroup:
                count60a += 1

                if totalbid <= 1:
                    count60b = count60b + paystrat[pairstrattag]
                else:
                    count60b = count60b + fbreakdowns60[paygroup]
                    
        fpayoffs60[idx80] = count60b/count60a
                
    return fpayoffs60


@numba.jit
def ppi_outgroup_im(fstrats7, foutimprob7, frandunif1010, foutpairings7, fnagents7):
    
    
    for idx90 in range(0, fnagents7):
        
        if frandunif1010[idx90] < foutimprob7:
            
            fstrats7[idx90][0:3] = fstrats7[foutpairings7[idx90]][0:3]
    
    return fstrats7


@numba.jit
def count_tag0(fstrats8):
    
    tag0count8 = 0
    
    for idx100 in range(0, len(fstrats8)):
        if fstrats8[idx100][-2] == 0:
            tag0count8 +=1
    
    tag0count8 = tag0count8/len(fstrats8)
    
    return tag0count8


@numba.jit
def ppi_10steps(fout_probN, fzerosN, fmutprobN, fbidsN, fumN, fnagentsN, fbreakdownsN, fstratsN, fpairings0N, fpairings1N, frandunif0N, frandunif2N, frandunif4N, frandunif6N, frandunif8N, frandunif10N, foutpairingsN):
    
    for idxN in range(0, 10):
        # calculate payoffs
        fpayoffsN = ppi_payoffs(fzerosN[idxN], fstratsN, fnagentsN, fbreakdownsN)
        # update strategies based on um
        fstratsN = ppi_update_withtags(fstratsN, fnagentsN, fumN, fpairings0N[idxN], fpairings1N[idxN], frandunif0N[idxN], fpayoffsN)
#         fstratsN = ppi_update_notags(fstratsN, fnagentsN, fumN, fpairings0N[idxN], fpairings1N[idxN], frandunif0N[idxN], fpayoffsN)
        # mutation if any
#         fstratsN = ppi_mutation_withTags(fstratsN, fmutprobN, frandunif2N[idxN], frandunif4N[idxN], frandunif6N[idxN], frandunif8N[idxN], fbidsN, fnagentsN)
        # outgroup immitation
        fstratsN = ppi_outgroup_im(fstratsN, fout_probN, frandunif10N[idxN], foutpairingsN[idxN], fnagentsN)
    
    return fstratsN

In [5]:
# use this cell to set parameters

high = 2/3
med = 1/2
low = 1/3

bids = np.asarray([low, med, high])
max_total = 1

mutprob = .001

outimm_prob = .01

numagents = 40
numtags = 2
tags = np.arange(numtags)

um = bids[-1]*numagents


numgroups = 2
groups = np.arange(numgroups)

proportion_of_types = np.asarray([0.5, 0.5])

breakdowns = np.asarray([0, 0])


timesteps = 3000
runs = 5000





In [6]:
# use this cell to run code


#measuring how long code takes to run
start = time.perf_counter()

fullcount = 0
paycount = 0
tag0count = 0

rng = np.random.default_rng()

for r in range(0, runs):
    
    strategies = make_strats(bids, numagents, numtags, proportion_of_types)
    
    for t in range(0, timesteps//10):
        zeros, pairings0, pairings1, randunif0, randunif2, randunif4, randunif6, randunif8, randunif10, outpairings = random10step(rng, numagents, proportion_of_types)
        strategies = ppi_10steps(outimm_prob, zeros, mutprob, bids, um, numagents, breakdowns, strategies, pairings0, pairings1, randunif0, randunif2, randunif4, randunif6, randunif8, randunif10, outpairings)
    
    fullcount += MATCHcount_disc(strategies, bids[-1])
    
    #get average payoff for powerful group
    zerosM = np.zeros(numagents, dtype = np.float64)
    final_pays = count_avg_payAGAINSTout(zerosM, strategies, numagents, breakdowns)
    powerpay, weakpay = count_avg_pay(strategies, final_pays)
    paycount += powerpay
    
    tag0count += count_tag0(strategies)
    
print("average 'discrimination' = ")
print(fullcount/(runs))
print(" ")

print("average powerful pay = ")
print(paycount/(runs))
print(" ")

print("average # of agents with tag0 = ")
print(tag0count/(runs))
print(" ")

finish = time.perf_counter()
print(f'Finished in {round(finish-start,0)/60} minutes')

average 'discrimination' = 
0.0003
 
average powerful pay = 
0.4869497499999994
 
average # of agents with tag0 = 
0.5015999999999999
 
Finished in 30.883333333333333 minutes


In [ ]:
# use this cell to test random things

# make_strats(bids, numagents, numtags, proportion_of_types)[0, 0:2]
# np.dtype(make_strats(bids, numagents, numtags, proportion_of_types)[0, 0])
# np.zeros(5, dtype=int)


# rng = np.random.default_rng(7563975)
# rightpairings = rng.permutation(numagents//2) + numagents//2
# print(rightpairings)


# ppi_payoffs(zeros, strategies, numagents, breakdowns)
# strategies
# zeros
# proportion_of_types[0]